In [60]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Specify the S3 Location of the Features

In [72]:
%store -r scikit_processing_job_s3_output_prefix

In [83]:
print('Previous Scikit Processing Job Name: {}'.format(scikit_processing_job_s3_output_prefix))

Previous Scikit Processing Job Name: sagemaker-scikit-learn-2020-03-30-03-34-18-188


In [84]:
prefix_train = '{}/output/bert-train-all'.format(scikit_processing_job_s3_output_prefix)
prefix_validation = '{}/output/bert-validation-all'.format(scikit_processing_job_s3_output_prefix)
prefix_test = '{}/output/bert-test-all'.format(scikit_processing_job_s3_output_prefix)

path_train = './{}'.format(prefix_train)
path_validation = './{}'.format(prefix_validation)
path_test = './{}'.format(prefix_test)

train_s3_uri = 's3://{}/{}'.format(bucket, prefix_train)
validation_s3_uri = 's3://{}/{}'.format(bucket, prefix_validation)
test_s3_uri = 's3://{}/{}'.format(bucket, prefix_test)

In [85]:
s3_input_train_data = sagemaker.s3_input(s3_data=train_s3_uri), distribution='ShardedByS3Key') 
s3_input_validation_data = sagemaker.s3_input(s3_data=validation_s3_uri), distribution='ShardedByS3Key')
s3_input_test_data = sagemaker.s3_input(s3_data=test_s3_uri), distribution='ShardedByS3Key')

print(s3_input_train_data.config)
print(s3_input_validation_data.config)
print(s3_input_test_data.config)

SyntaxError: invalid syntax (<ipython-input-85-93b3c10d512f>, line 1)

In [86]:
!cat src_bert_tf2/tf_bert_reviews.py

import time
import random
import pandas as pd
from glob import glob
import argparse
import json
import subprocess
import sys
import os
import tensorflow as tf
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'tensorflow==2.0.0'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'transformers'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'sagemaker-tensorflow==2.0.0.1.1.0'])
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import TextClassificationPipeline
from transformers.configuration_distilbert import DistilBertConfig

MAX_SEQ_LENGTH = 128
BATCH_SIZE = 128 
EVAL_BATCH_SIZE = BATCH_SIZE * 2
EPOCHS = 10 
STEPS_PER_EPOCH = 1000
VALIDATION_STEPS = 1000
CLASSES = [1, 2, 3, 4, 5]
# XLA is an optimization compiler for tensorflow
USE_XLA = False 
# Mixed precision can help to speed up training time
USE_AMP = False


def select_data_and_label_from_record(record):
    

In [87]:
from sagemaker.tensorflow import TensorFlow

model_output_path = 's3://{}/models/tf2-bert'.format(bucket)

bert_estimator = TensorFlow(entry_point='tf_bert_reviews.py',
                            source_dir='src_bert_tf2',
                            role=role,
                            train_instance_count=2, # Make sure you have at least this number of input files or the ShardedByS3Key distibution strategy will fail the job due to no data available
                            train_instance_type='ml.p3.16xlarge',
                            py_version='py3',
                            framework_version='2.0.0',
                            output_path=model_output_path,
#                            hyperparameters={'model_type':'bert',
#                                             'model_name': 'bert-base-cased'},
                            distributions={'parameter_server': {'enabled': True}},
                            enable_cloudwatch_metrics=True,
                            input_mode='Pipe'
                           )

# Train the model

In [88]:
bert_estimator.fit(inputs={'train': s3_input_train_data, 
                           'validation': s3_input_validation_data,}, 
                   wait=False)

In [89]:
training_job_name = bert_estimator.latest_training_job.name
print('training_job_name:  {}'.format(training_job_name))

training_job_name:  tensorflow-training-2020-04-04-07-42-44-091


In [90]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/{}">Training Job</a> After About 5 Minutes</b>'.format(region, training_job_name)))


In [91]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/TrainingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(region, training_job_name)))


In [92]:
from IPython.core.display import display, HTML

training_job_s3_output_prefix = 'models/tf2-bert/{}'.format(training_job_name) # 'models/tf-bert/script-mode/training-runs/{}'.format(training_job_name)

display(HTML('<b>Review <a href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Training Job Has Completed</b>'.format(bucket, training_job_s3_output_prefix, region)))


# Download and Load the Trained Model

In [ ]:
# download the model artifact from AWS S3

!aws s3 cp $model_output_path/$training_job_name/output/model.tar.gz ./models

#!aws s3 cp s3://sagemaker-us-east-1-835319576252/models/tf-bert/script-mode/training-runs/tensorflow-training-2020-03-24-04-41-39-405/output/model.tar.gz ./models/tf2-bert/

In [ ]:
import tarfile
import pickle as pkl

tar = tarfile.open('./models/model.tar.gz')
tar.extractall(path='./models')
tar.close()

In [ ]:
!ls -al ./models

In [ ]:
# Must upgrade wrapt before installing TF
!pip install -q pip --upgrade
!pip install -q wrapt --upgrade --ignore-installed
!pip install -q tensorflow==2.0.0

# Load the model

In [ ]:
# TODO